# CryptoClustering Notebook

---

## Setup

### Import python and file dependencies

In [1]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Custom dependencies
from lib import make_scaled_df, elbow_curve, modeling

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    'Resources/crypto_market_data.csv',
    index_col='coin_id')

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


### Basic data visualization

In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width = 800,
    height = 400,
    rot = 90,
    title = 'Market Data Over Time'
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

## Modeling - KMeans

### Data preparation

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data)

In [6]:
scaled_df = make_scaled_df(market_data_scaled, df_market_data.columns, df_market_data.index)
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


### Elbow Curve

In [7]:
org_elbow = elbow_curve(11, scaled_df)
org_elbow

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4 would most likely be the correct answer, however depending on how the analysis of the clusters turns out it could be anywhere from 3-5  
  
  

### Model creation

In [8]:
market_prediction_df, plot = modeling(4, scaled_df)
display(market_prediction_df.head())
plot

Clusters: 
[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,CryptoCluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


:NdOverlay   [CryptoCluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

## Modeling - PCA & KMeans

### Data preparation

In [9]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)

# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
scaled_pca = pca.fit_transform(scaled_df)

# View the scaled PCA data
scaled_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [10]:
# Raw array
explained_var = pca.explained_variance_ratio_
explained_var

array([0.3719856 , 0.34700813, 0.17603793])

In [11]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
print('Data per Primary Component')
[print(f'PCA {num}: {round(percent * 100, 3)}%') for num, percent in enumerate(explained_var, 1)]

print('\nTotal Data Kept : Lost')
kept = explained_var.sum() * 100
loss = 100 - kept
print(f'{round(kept, 3)}% : {round(loss, 3)}%')

Data per Primary Component
PCA 1: 37.199%
PCA 2: 34.701%
PCA 3: 17.604%

Total Data Kept : Lost
89.503% : 10.497%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

In [12]:
# Display sample of scaled PCA dataframe
pca_df = make_scaled_df(scaled_pca, ['PCA1', 'PCA2', 'PCA3'], df_market_data.index)
pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


### Elbow Curve

In [13]:
pca_elbow = elbow_curve(11, pca_df)
pca_elbow

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** It's even more obviously 4 this time


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Only that it seems clearer that 4 is the best k value

### Model creation

In [14]:
pca_market_prediction_df, pca_plot = modeling(4, pca_df)
display(pca_market_prediction_df.head())
pca_plot

Clusters: 
[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


,PCA1,PCA2,PCA3,CryptoCluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


:NdOverlay   [CryptoCluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

---

## Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [15]:
# Composite plot to contrast the Elbow curves
org_elbow * pca_elbow

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [16]:
# Composite plot to contrast the clusters
plot * pca_plot

:Overlay
   .NdOverlay.I  :NdOverlay   [CryptoCluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [CryptoCluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

In [17]:
plot

:NdOverlay   [CryptoCluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [18]:
pca_plot

:NdOverlay   [CryptoCluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** When clustering using fewer features, the data is separated with more linearity and with more distinctive categorization. There is slightly less data (ie. data loss) and the features themselves require some distinguishing, but the points are easier to understand why they've been grouped together.